Between May 24th and May 28th [Anastasios (Tasos) Sidiropoulos](https://sidiropo.people.uic.edu/) provided several counter examples indicating that [our constraints for the the polytope of wealth distributions](https://github.com/renepickhardt/Lightning-Network-Limitations/blob/906b3a4d691cbfb30c2f385c6697b4fe2bb81676/Limits%20of%20two%20party%20channels/Geometric%20Argument%20for%20Bimodal%20Liquidity%20Distribution%20in%20Payment%20Channels%20of%20the%20Lightning%20Network.ipynb) are too weak. In particular he provided [a point in our polytope for which no feasible lighting network state existed](https://twitter.com/sidiropo/status/1795458687500239055)

His example assumes the following network

```
(A)---2---(B)---2---(C)---2---(D)---2---(E)---2---(F)
```
where `A,B,C,F` hold `1` coin each and `D,E` hold `3`coins. 

one can easily check that $w_i+w_j>=c_{ij}$ and no user has more coins than total capacity of the neighboring channels. 

However no valid state for this network exists that produces this particular wealth distribution. This means our polytope of wealth distributions is too large and the formulation of inequalities cannot hold. 

## Smaller example

The example of Sidiropoulos can actually be made smaller. Consider the following network: 
```
(A)---2---(B)---2---(C)---2---(D)---2---(E)
```
where `A,B,C` hold `1` coin each and `D` holds `3` coins and `E` holds `2` coins. 

Again no network state exists that would produce this wealth distribution. 

## Goal of this notebook

Leaving the [formulation of inequalities to describe the polytope of wealth distributions]((https://github.com/renepickhardt/Lightning-Network-Limitations/blob/906b3a4d691cbfb30c2f385c6697b4fe2bb81676/Limits%20of%20two%20party%20channels/Geometric%20Argument%20for%20Bimodal%20Liquidity%20Distribution%20in%20Payment%20Channels%20of%20the%20Lightning%20Network.ipynb)) and focussing on the polytope $S$ of channel states and the natural mapping from $\omega: S\rightarrow S/\sim \cong W$ one is interested if for $w\in S/\sim$ we can construct and element of $\omega^{-1}(\{w\})$ (aka an element in the preimage of $w$)

We claim that a preimage exists if the following system of linear equations has a non trivial solution. This can be tested by solving the large or small linear program constructed and provided in this notebook. The system of linear equations is described in the code! The gist for the two programs is as follows:

* $m$ = number of edges
* $n$ = number of peers on the network

### Large linear program 
* $2m$ variables encoding the wealth of each user in the channel (this can probably be reduced to $m$ variables)
* $m$ constraints with conservation of liquidity in channels
* $n$ constraints relating to the wealth distribution of $n$ peers
* $1$ constraint relating to the total number of coins
* $2m+1$ bounds actually $m$ bounds but each is twice and 1 bound for total number of coins
  
### Small linear program
* $m$ variables. One representing the liquidity of the smaller node id for each channel
* $n$ constraints comming from the wealth of each user
Conservation of liquidity in channels is being used 
* $m$ bounds as the liquidity in each channel must be between `0` and `cap`


## Acknowledgements

Obviously a big thanks goes to [Anastasios (Tasos) Sidiropoulos](https://sidiropo.people.uic.edu/) for the initial counter example and the discussion on twitter. Also to Stefan Richter for putting out the hypothesis in the thread that a cycle base would be sufficient to test for conservation of liquidity. The work is sponsored through a [grant from OpenSats](https://opensats.org/blog/rene-pickhardt-receives-lts-grant) and through [individual patreons](https://www.patreon.com/renepickhardt).

In [1]:
from scipy.optimize import linprog

Create channels and wealth distribution according to the modified counter example from Sidiropoulos

In [2]:
channels = {(0,1):2, (1,2):2, (2,3):2, (3,4):2}
wealth = {0:1,1:1,2:1,3:3,4:2}

def print_network(channels):
    print("Given the following network:\n")
    for chan, cap in channels.items():
        print("({})-?-----|{}|-----?-({})".format(chan[0],cap,chan[1]))
print_network(channels)

Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|2|-----?-(2)
(2)-?-----|2|-----?-(3)
(3)-?-----|2|-----?-(4)


Construction of the system of $n+m+1$ linear equations in $2m$ variables ($w_{ij}$) and the corresponding linear program that must be feasible in order for a lighting network channel state to exist that produces the wealth distribution: 

* $w_{ij} + w_{ji} = c_{ij} \forall i<j$ ($m$ constraints)
* $\sum_{j=1}^nw_{ij} = w_i$ ($n$ constraints)
* $\sum_{i,j}w_{ij} = \sum_{i,j}c_{ij}$

This programm depends on $m+n$ parameters ($c_{ij}$ and $w_i$}

A feasible solution means that we have a non trivial preimage for a given wealth distribution



In [3]:
def get_state_large_lp(channels, wealth):
    num_coins = sum(channels.values())
    m = len(channels)
    n = len(wealth)
    
    channel_idx = {}
    A = []
    b = []
    
    wealth_constraints = []
    for i in range(n):
        row = [0]*2*m
        wealth_constraints.append(row)
    
    bounds = []
    for k, chan in enumerate(channels.items()):
        row = [0]*2*m
        row[2*k]=1
        row[2*k+1]=1
        A.append(row)
        key,cap = chan
        #add bounds of channel twice as each channel adds two variables
        bounds.append((0,cap))
        bounds.append((0,cap))
    
        x,y = key
        channel_idx[2*k]=key
        channel_idx[2*k+1]=(y,x)
        
        wealth_constraints[x][2*k]=1
        wealth_constraints[y][2*k+1]=1
        b.append(cap)
    for k, row in enumerate(wealth_constraints):
        A.append(row)
        b.append(wealth[k])
    A.append([1]*2*m)
    num_coins=sum(wealth.values())
    b.append(num_coins)
    
    c = [1]*2*m
    
    #print(A,b,bounds)
    print_network(channels)
    res = linprog(c, A_eq=A, b_eq=b, bounds=bounds)
    if res.success == True:
        print("\nthis state vector satisfies the wealth distribution {}".format(wealth))
        for i,x in enumerate(res.x):
            print(channel_idx[i], x)
    else:
        print("\nThe wealth distribution {} is infeasible for the network".format(wealth))

get_state_large_lp(channels, wealth)

Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|2|-----?-(2)
(2)-?-----|2|-----?-(3)
(3)-?-----|2|-----?-(4)

The wealth distribution {0: 1, 1: 1, 2: 1, 3: 3, 4: 2} is infeasible for the network


## Alternative way that uses only $m$ variabales and $n$ constraints. 

We observe that the conservation of liquidity constraints can be utilized to replace $m$ variables: 

$w_{ij}=c_{ij}-w_{ji} \Leftrightarrow w_{ji}=c_{ij}-w_{ij}$

Thus we can allways assume that $i<j$. Recall the constraints for the wealth $w_i$ of user $i$ used to be $w_i = \sum_{j=1}^nw_{ij}$ This will now become:

$w_i = \sum_{j=1}^{i}(c_{ij}-w_{ji}) + \sum_{j=i+1}^nw_{ij}$

$\Leftrightarrow w_i - \sum_{j=1}^{i}c_{ij} = \sum_{j=i+1}^nw_{ij} -\sum_{j=1}^{i}w_{ji}$

$\Leftrightarrow (\sum_{j=1}^{i}c_{ij}) - w_i = (\sum_{j=1}^{i}w_{ji}) - (\sum_{j=i+1}^nw_{ij})$

In particular the constraint that the total number of coins needs to be constant can be removed as it is implicitley given by using the capacities on the right hand side of the system of linear equations

In [4]:
def get_state_small_lp(channels, wealth):
    num_coins = sum(channels.values())
    m = len(channels)
    n = len(wealth)
    
    channel_idx = {}
    A = []
    b = []
    
    wealth_constraints = {i:[0]*m for i in range(n)}
    bounds = []
    for k, chan in enumerate(channels.items()):
        key,cap = chan
        bounds.append((0,cap))
        channel_idx[k]=key
    
        x,y = key
        #requirement to remove variables channels are always going from lower to higher id
        assert x < y
        
        wealth_constraints[x][k]=-1
        wealth_constraints[y][k]=1
    for i, row in wealth_constraints.items():
        A.append(row)
        cap = 0
        for j in range(i):
            if (j,i) in channels:
                cap+=channels[(j,i)]
        b.append(cap - wealth[i])
    
    c = [1]*m
    
    #print(A,b,bounds)
    
    res = linprog(c, A_eq=A, b_eq=b, bounds=bounds)

    print_network(channels)
        
    if res.success == True:
        print("\n\nThe following state the wealth distribution {}:\n".format(wealth))
        for i,x in enumerate(res.x):
            cap = channels[channel_idx[i]]
            print("({})-{}-----|{}|-----{}-({})".format(channel_idx[i][0],x,cap,cap-x,channel_idx[i][1]))
            
    else:
        print("\nThe wealth distribution {} is infeasible for the network".format(wealth))
get_state_small_lp(channels, wealth)

Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|2|-----?-(2)
(2)-?-----|2|-----?-(3)
(3)-?-----|2|-----?-(4)

The wealth distribution {0: 1, 1: 1, 2: 1, 3: 3, 4: 2} is infeasible for the network


In [5]:
channels = {(0,1):2, (1,2):2, (2,3):2, (3,4):2}
wealth = {0:1,1:2,2:1,3:2,4:2}

get_state_large_lp(channels, wealth)
get_state_small_lp(channels, wealth)

Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|2|-----?-(2)
(2)-?-----|2|-----?-(3)
(3)-?-----|2|-----?-(4)

this state vector satisfies the wealth distribution {0: 1, 1: 2, 2: 1, 3: 2, 4: 2}
(0, 1) 1.0
(1, 0) 1.0
(1, 2) 1.0
(2, 1) 1.0
(2, 3) -0.0
(3, 2) 2.0
(3, 4) -0.0
(4, 3) 2.0
Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|2|-----?-(2)
(2)-?-----|2|-----?-(3)
(3)-?-----|2|-----?-(4)


The following state the wealth distribution {0: 1, 1: 2, 2: 1, 3: 2, 4: 2}:

(0)-1.0-----|2|-----1.0-(1)
(1)-1.0-----|2|-----1.0-(2)
(2)-0.0-----|2|-----2.0-(3)
(3)-0.0-----|2|-----2.0-(4)


In [6]:
channels = {(0,1):2, (1,2):3, (0,2):10}
wealth = {0:3,1:5,2:6}

get_state_large_lp(channels, wealth)
get_state_small_lp(channels, wealth)


Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|3|-----?-(2)
(0)-?-----|10|-----?-(2)

The wealth distribution {0: 3, 1: 5, 2: 6} is infeasible for the network
Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|3|-----?-(2)
(0)-?-----|10|-----?-(2)

The wealth distribution {0: 3, 1: 5, 2: 6} is infeasible for the network


In [7]:
channels = {(0,1):2, (1,2):3, (0,2):10}
wealth = {0:3,1:4,2:8}

get_state_large_lp(channels, wealth)
get_state_small_lp(channels, wealth)

Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|3|-----?-(2)
(0)-?-----|10|-----?-(2)

this state vector satisfies the wealth distribution {0: 3, 1: 4, 2: 8}
(0, 1) -0.0
(1, 0) 2.0
(1, 2) 2.0
(2, 1) 1.0
(0, 2) 3.0
(2, 0) 7.0
Given the following network:

(0)-?-----|2|-----?-(1)
(1)-?-----|3|-----?-(2)
(0)-?-----|10|-----?-(2)


The following state the wealth distribution {0: 3, 1: 4, 2: 8}:

(0)-0.0-----|2|-----2.0-(1)
(1)-2.0-----|3|-----1.0-(2)
(0)-3.0-----|10|-----7.0-(2)
